### **Práctica 2: Conteo de vehículos que transitan en vías de tráfico**

***Autores:** Mendoza Peña, Raúl y Casimiro Torres, Kimberly*

<img src="./Imagen_conteo_vehiculos.png" alt="Imagen con el conteo de vehículos realizado" width="1000"/>

#### **Descripción del Proyecto**

Este proyecto consiste en desarrollar un sistema que permita contar vehículos en un video de tráfico (`tráfico01.mp4`). El objetivo es detectar y contar vehículos que cruzan diferentes líneas de monitoreo, correspondientes a carriles o vías en el video.

#### **1. Importación de bibliotecas necesarias**

Para este proyecto, utilizamos únicamente la biblioteca **`cv2`** de OpenCV.

##### **¿Qué es OpenCV?**
OpenCV es una biblioteca de código abierto muy popular para el procesamiento de imágenes y videos en tiempo real. Proporciona herramientas poderosas para detectar y rastrear objetos, realizar transformaciones en imágenes, manipular videos y más. En este proyecto, OpenCV se utiliza para:

- Leer y procesar el video de tráfico (`tráfico01.mp4`).
- Aplicar técnicas de segmentación para separar objetos en movimiento (vehículos) del fondo.
- Detectar contornos y rastrear vehículos.
- Visualizar los resultados en tiempo real dibujando líneas de monitoreo y mostrando conteos acumulados.

In [ ]:
import cv2

#### **2. Configuración del video**
El video de tráfico que vamos a analizar se encuentra en un archivo llamado `tráfico01.mp4`. La ruta del archivo se define en la variable video_path. Este será el punto de partida para que OpenCV pueda cargar el video y procesarlo.

In [ ]:
# Ruta del video proporcionado
video_path = 'tráfico01.mp4'

#### **3. Coordenadas de las líneas de monitoreo**
Definimos un conjunto de líneas que sirven para monitorear diferentes carriles o vías en el video. Estas líneas se especifican mediante sus coordenadas en el espacio de píxeles del frame del video. Cada línea está representada por un diccionario con los siguientes valores:

-  **x1, y1:** Coordenadas del punto inicial de la línea.
-  **x2, y2:** Coordenadas del punto final de la línea.

In [ ]:
# Coordenadas de las líneas segmentadas (Vías y Carriles separados)
lines_positions = [
    {"x1": 480, "y1": 800, "x2": 570, "y2": 800},  # Vía 1
    {"x1": 600, "y1": 800, "x2": 700, "y2": 800},  # Vía 2
    {"x1": 990, "y1": 750, "x2": 1090, "y2": 750},  # Carril 1 -> Vía 3
    {"x1": 1280, "y1": 890, "x2": 1500, "y2": 890},  # Carril 2 -> Vía 3
    {"x1": 1200, "y1": 660, "x2": 1260, "y2": 660},  # Carril 3 -> Vía 4
    {"x1": 1450, "y1": 750, "x2": 1550, "y2": 750},  # Carril 4 -> Vía 4
    {"x1": 1650, "y1": 790, "x2": 1800, "y2": 790},  # Carril 5 -> Vía 4
    {"x1": 1500, "y1": 450, "x2": 1800, "y2": 450},  # Vía 5
]

Las coordenadas están ajustadas específicamente al video tráfico01.mp4 para que las líneas coincidan visualmente con las posiciones de los carriles y vías en el frame. Los valores se definen en píxeles, que es la unidad de medida en OpenCV para los frames de video. Estas líneas actúan como "sensores" virtuales para detectar el paso de vehículos.

#### **4: Configuración de parámetros para la detección de vehículos**
En esta sección, configuramos los parámetros clave que controlan cómo se detectan y rastrean los vehículos en el video.

##### Parámetros incluidos:
1. **Umbrales mínimos de área**: Controlan el tamaño mínimo necesario para que un objeto sea considerado un vehículo.
2. **Ancho máximo de vehículos**: Filtra objetos que excedan un tamaño razonable.
3. **Listas y diccionarios para el conteo y seguimiento**: Incluyen contadores de vehículos, objetos rastreados y su tiempo de expiración.


In [ ]:
# Umbrales mínimos de área y tamaños para cada carril
area_thresholds = [400, 500, 700, 800, 700, 700, 500, 600]  # Configuración específica para cada carril

# Ancho máximo permitido para un vehículo
max_vehicle_width = 400  

# Contadores para el número de vehículos detectados por carril
vehicle_count = [0] * len(lines_positions)  # Lista de contadores inicializados en 0

# Diccionarios para rastrear objetos detectados por carril
tracked_objects = [{} for _ in range(len(lines_positions))]  # Lista de diccionarios vacíos

# Identificadores únicos para los objetos detectados en cada carril
object_ids = [0] * len(lines_positions)  # Inicializados en 0 para cada carril

# Tiempo máximo que un objeto puede estar inactivo antes de considerarse "expirado"
object_expiry = 16  # Definido en número de frames

#### **5: Función auxiliar para detectar cruces de líneas**
Esta función verifica si un objeto detectado (representado por un punto) cruza una línea específica en el espacio de píxeles del video.

##### Parámetros de entrada:
1. **`cx`**: Coordenada X del centroide del objeto.
2. **`cy`**: Coordenada Y del centroide del objeto.
3. **`line`**: Diccionario que define una línea en el espacio de píxeles, con los campos:
   - `x1`, `y1`: Coordenadas del punto inicial de la línea.
   - `x2`, `y2`: Coordenadas del punto final de la línea.

##### Salida:
- Devuelve `True` si el centroide del objeto se encuentra dentro del rango horizontal de la línea (`x1` a `x2`) y si la distancia vertical al centroide desde la línea (`cy - y1`) es menor o igual a 10 píxeles.  
- De lo contrario, devuelve `False`.

In [ ]:
def crosses_line(cx, cy, line):  
    # Define la función `crosses_line` para verificar si un objeto cruza una línea específica.

    return line["x1"] <= cx <= line["x2"] and abs(cy - line["y1"]) <= 10  
    # Comprueba si el objeto está dentro del rango horizontal de la línea (`x1` <= `cx` <= `x2`)
    # y si la distancia vertical entre el centro del objeto (`cy`) y la línea (`y1`) es menor o igual a 10 píxeles.

#### **6. Carga del video y configuración del sustractor de fondo**
En esta sección, cargamos el video proporcionado y configuramos el sustractor de fondo para detectar objetos en movimiento (vehículos).

##### Pasos realizados:
1. Abrimos el archivo de video con OpenCV.
2. Verificamos si el video se ha cargado correctamente.
3. Configuramos un **sustractor de fondo**, una herramienta que separa los objetos en movimiento del fondo estático.
4. Parámetros del sustractor:
    - *history=500*: Número de frames usados para construir el modelo del fondo.
    - *varThreshold=50*: Sensibilidad para detectar cambios en los píxeles. Valores más bajos hacen que el modelo sea más sensible, mientras que valores más altos lo hacen menos sensible.
    - *detectShadows=False*: Desactiva la detección de sombras para evitar falsos positivos.
5. Inicializamos un contador de frames para rastrear el progreso del video.

In [ ]:
cap = cv2.VideoCapture(video_path)  
# Abre el archivo de video especificado en la ruta `video_path` para procesarlo.

if not cap.isOpened():  
    # Verifica si el archivo de video se abrió correctamente.

    print("Error: No se puede abrir el video.")  
    # Muestra un mensaje de error en caso de que el video no pueda abrirse.
else:  
    # Ejecuta este bloque si el archivo de video se abrió correctamente.

    background_subtractor = cv2.createBackgroundSubtractorMOG2(history=500, varThreshold=50, detectShadows=False)  
    # Configura un sustractor de fondo para detectar objetos en movimiento:
    # - `history=500`: Usa los últimos 500 frames para construir el modelo del fondo.
    # - `varThreshold=50`: Define la sensibilidad al cambio de píxeles (valores más bajos detectan más).
    # - `detectShadows=False`: No considera sombras para evitar falsos positivos.

    frame_count = 0  
    # Inicializa el contador de frames en 0 para rastrear el progreso del video.

#### **6. Procesamiento del video frame por frame**
En esta sección, procesamos cada frame del video para detectar objetos en movimiento (vehículos) utilizando el sustractor de fondo configurado anteriormente. Este proceso incluye convertir los frames a escala de grises, aplicar el sustractor de fondo y extraer contornos de los objetos detectados.

##### Pasos realizados:
1. **Lectura del frame actual**: Se obtiene el siguiente frame del video.
2. **Conversión a escala de grises**: Simplifica la imagen para el procesamiento.
3. **Sustracción de fondo**: Detecta las diferencias entre el fondo estático y los objetos en movimiento.
4. **Operaciones morfológicas**: Limpia la máscara de ruido o pequeños fragmentos.
5. **Binarización**: Asegura que solo se detecten objetos relevantes.
6. **Detección de contornos**: Identifica los límites de los objetos detectados.

In [ ]:
while True:  
    # Inicia un bucle infinito para procesar cada frame del video.

    ret, frame = cap.read()  
    # Lee el siguiente frame del video.
    # `ret` indica si la lectura fue exitosa (True) o si se alcanzó el final (False).
    # `frame` contiene los datos del frame actual.

    if not ret:  
        # Si no se pudo leer el frame (por ejemplo, al llegar al final del video):
        break  
        # Sale del bucle para detener el procesamiento.

    frame_count += 1  
    # Incrementa el contador de frames en 1 para rastrear el número de frames procesados.

    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)  
    # Convierte el frame de color (BGR) a escala de grises para simplificar el procesamiento.

    fg_mask = background_subtractor.apply(gray)  
    # Aplica el sustractor de fondo al frame en escala de grises para detectar objetos en movimiento.

    fg_mask = cv2.morphologyEx(fg_mask, cv2.MORPH_CLOSE, cv2.getStructuringElement(cv2.MORPH_RECT, (5, 5)))  
    # Aplica una operación de cierre (closing) para eliminar ruido y pequeños huecos en la máscara.

    _, fg_mask = cv2.threshold(fg_mask, 254, 255, cv2.THRESH_BINARY)  
    # Convierte la máscara en una imagen binaria (solo valores 0 y 255) para resaltar objetos detectados.

    contours, _ = cv2.findContours(fg_mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)  
    # Encuentra los contornos en la máscara binaria:
    # - `cv2.RETR_EXTERNAL`: Obtiene solo los contornos externos (ignora los internos).
    # - `cv2.CHAIN_APPROX_SIMPLE`: Reduce los puntos del contorno para ahorrar memoria.


#### **7. Procesamiento de contornos y conteo de vehículos**
En esta sección, procesamos los contornos detectados para identificar vehículos, determinar si cruzan las líneas de monitoreo y actualizar los contadores. Este análisis incluye:

1. **Extracción de propiedades del contorno**: Calculamos el centroide y el tamaño del objeto.
2. **Filtrado de objetos irrelevantes**: Eliminamos sombras y objetos pequeños.
3. **Detección del carril correspondiente**: Verificamos si el objeto cruza alguna línea de monitoreo.
4. **Rastreo de vehículos**: Identificamos objetos nuevos o actualizamos objetos existentes.
5. **Actualización de contadores**: Incrementamos el conteo de vehículos cuando un nuevo objeto cruza la línea.


In [ ]:
for contour in contours:  
    # Itera sobre cada contorno detectado en la máscara binaria.

    x, y, w, h = cv2.boundingRect(contour)  
    # Calcula el rectángulo delimitador del contorno:
    # - `x`, `y`: Coordenadas de la esquina superior izquierda.
    # - `w`, `h`: Ancho y alto del rectángulo.

    cx, cy = x + w // 2, y + h // 2  
    # Calcula el centro del rectángulo delimitador (`cx`, `cy`).

    if w > max_vehicle_width or cv2.contourArea(contour) < 500:  
        # Filtra objetos irrelevantes:
        # - Si el ancho del objeto excede `max_vehicle_width`, lo ignora.
        # - Si el área del contorno es menor a 500 píxeles, también lo ignora.
        continue  
        # Salta al siguiente contorno.

    for i, line in enumerate(lines_positions):  
        # Itera sobre todas las líneas de monitoreo definidas.

        if crosses_line(cx, cy, line):  
            # Verifica si el centro del objeto cruza la línea actual.

            if cv2.contourArea(contour) < area_thresholds[i]:  
                # Verifica si el área del contorno es menor al umbral mínimo para esa línea.
                # Si no cumple, el contorno se ignora.
                continue  

            new_object = True  
            # Marca inicialmente el objeto como "nuevo".

            for obj_id in list(tracked_objects[i].keys()):  
                # Itera sobre los objetos ya rastreados en el carril correspondiente.

                prev_cx, prev_cy, last_seen = tracked_objects[i][obj_id]  
                # Recupera las coordenadas previas y el último frame donde fue visto el objeto.

                if abs(cx - prev_cx) < 50 and abs(cy - prev_cy) < 50:  
                    # Comprueba si el objeto actual está cerca de un objeto rastreado previamente
                    # (dentro de un rango de 50 píxeles en ambas coordenadas).

                    tracked_objects[i][obj_id] = (cx, cy, frame_count)  
                    # Actualiza las coordenadas y el último frame visto del objeto rastreado.

                    new_object = False  
                    # Marca el objeto como "no nuevo" porque ya está siendo rastreado.
                    break  

            if new_object:  
                # Si el objeto es realmente nuevo:
                tracked_objects[i][object_ids[i]] = (cx, cy, frame_count)  
                # Lo agrega a la lista de objetos rastreados para el carril correspondiente.

                vehicle_count[i] += 1  
                # Incrementa el conteo de vehículos en el carril correspondiente.

                object_ids[i] += 1  
                # Actualiza el próximo identificador único disponible para objetos nuevos.

#### **8. Limpieza de objetos expirados y visualización de resultados**
En esta sección, realizamos dos tareas principales:

1. **Limpieza de objetos expirados**: Eliminamos del rastreo aquellos objetos que no han sido detectados en un tiempo prolongado (basado en el número de frames).
2. **Visualización de resultados**: Dibujamos las líneas de monitoreo y mostramos el conteo acumulado de vehículos directamente sobre los frames procesados.

In [ ]:
for i, _ in enumerate(lines_positions):  
    # Itera sobre cada línea de monitoreo definida en `lines_positions`.

    expired_ids = [  
        # Crea una lista de identificadores (`expired_ids`) para los objetos que han expirado.

        obj_id for obj_id, (_, _, last_seen) in tracked_objects[i].items()  
        # Recorre todos los objetos rastreados (`tracked_objects[i]`) en el carril actual.
        # Extrae el identificador del objeto (`obj_id`) y el último frame en el que fue visto (`last_seen`).

        if frame_count - last_seen > object_expiry  
        # Comprueba si el tiempo transcurrido desde el último frame en que se vio el objeto
        # excede el límite (`object_expiry`). Si es así, el objeto se considera expirado.
    ]

    for obj_id in expired_ids:  
        # Itera sobre los identificadores de los objetos expirados.

        del tracked_objects[i][obj_id]  
        # Elimina cada objeto expirado del diccionario de objetos rastreados.

    for i, line in enumerate(lines_positions):  
        # Itera nuevamente sobre todas las líneas de monitoreo.

        if 2 <= i <= 6:  
            # Comprueba si el índice de la línea está entre 2 y 6 (corresponde a carriles específicos).

            label = f"Carril {i - 1}: {vehicle_count[i]}"  
            # Asigna una etiqueta que muestra el conteo de vehículos para los carriles.
        else:  
            # Si la línea no está entre los índices 2 y 6, se trata de una vía completa.

            label = f"Via {i + 1}: {vehicle_count[i]}"  
            # Asigna una etiqueta que muestra el conteo de vehículos para las vías.

        cv2.line(frame, (line["x1"], line["y1"]), (line["x2"], line["y2"]), (0, 255, 0), 2)  
        # Dibuja la línea de monitoreo en el frame:
        # - Usa las coordenadas (`x1`, `y1`, `x2`, `y2`) de la línea actual.
        # - Color: verde `(0, 255, 0)`.
        # - Grosor: 2 píxeles.

        cv2.putText(frame, label, (line["x1"], line["y1"] - 20),  
                    cv2.FONT_HERSHEY_SIMPLEX, 0.6, (255, 255, 255), 2)  
        # Dibuja la etiqueta con el conteo de vehículos encima de la línea:
        # - Texto: el valor de `label`.
        # - Posición: ligeramente encima de la línea (`line["y1"] - 20`).
        # - Fuente: `cv2.FONT_HERSHEY_SIMPLEX`.
        # - Tamaño de fuente: 0.6.
        # - Color: blanco `(255, 255, 255)`.
        # - Grosor del texto: 2 píxeles.

    cv2.imshow("Vehicle Counting", frame)  
    # Muestra el frame procesado en una ventana llamada "Vehicle Counting".

    if cv2.waitKey(30) & 0xFF == ord('q'):  
        # Espera 30 milisegundos para detectar si se presiona una tecla.
        # Si la tecla presionada es `q`, sale del bucle.

        break  
        # Rompe el bucle para detener el procesamiento.

cap.release()  
# Libera los recursos asociados al archivo de video.

cv2.destroyAllWindows()  
# Cierra todas las ventanas abiertas por OpenCV.

#### **9. Resultados finales**
En esta última sección, mostramos los conteos finales de vehículos para cada carril o vía después de procesar el video completo. Este paso simplemente imprime los resultados acumulados en la consola.

In [ ]:
for i, count in enumerate(vehicle_count):  
    # Itera sobre la lista `vehicle_count`, donde:
    # - `i` es el índice del carril o vía.
    # - `count` es el número total de vehículos contados para ese carril o vía.

    if 2 <= i <= 6:  # Carriles de las vías 3 y 4  
        # Comprueba si el índice corresponde a los carriles de las vías 3 y 4 (índices entre 2 y 6).

        print(f"Carril {i - 1}: {count} vehículos contados.")  
        # Imprime el conteo de vehículos para los carriles, ajustando el índice para que comience desde 1.

    else:  
        # Si el índice no corresponde a los carriles de las vías 3 y 4.

        print(f"Via {i + 1}: {count} vehículos contados.")  
        # Imprime el conteo de vehículos para las vías completas, ajustando el índice para que comience desde 1.

## **Código completo - Vídeo Normal**

In [3]:
import cv2

video_path = 'tráfico01.mp4'

lines_positions = [
    {"x1": 480, "y1": 800, "x2": 570, "y2": 800},
    {"x1": 600, "y1": 800, "x2": 700, "y2": 800},
    {"x1": 990, "y1": 750, "x2": 1090, "y2": 750},
    {"x1": 1280, "y1": 890, "x2": 1500, "y2": 890},
    {"x1": 1200, "y1": 660, "x2": 1260, "y2": 660},
    {"x1": 1450, "y1": 750, "x2": 1550, "y2": 750},
    {"x1": 1650, "y1": 790, "x2": 1800, "y2": 790},
    {"x1": 1500, "y1": 450, "x2": 1800, "y2": 450},
]

area_thresholds = [400, 500, 700, 800, 700, 700, 500, 600]
max_vehicle_width = 400

vehicle_count = [0] * len(lines_positions)
tracked_objects = [{} for _ in range(len(lines_positions))]
object_ids = [0] * len(lines_positions)
object_expiry = 16

def crosses_line(cx, cy, line):
    """Verifica si un punto cruza una línea específica."""
    return line["x1"] <= cx <= line["x2"] and abs(cy - line["y1"]) <= 10

cap = cv2.VideoCapture(video_path)

if not cap.isOpened():
    print("Error: No se puede abrir el video.")
else:
    background_subtractor = cv2.createBackgroundSubtractorMOG2(history=500, varThreshold=50, detectShadows=False)
    frame_count = 0

    while True:
        ret, frame = cap.read()
        if not ret:
            break

        frame_count += 1
        gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
        fg_mask = background_subtractor.apply(gray)
        fg_mask = cv2.morphologyEx(fg_mask, cv2.MORPH_CLOSE, cv2.getStructuringElement(cv2.MORPH_RECT, (5, 5)))
        _, fg_mask = cv2.threshold(fg_mask, 254, 255, cv2.THRESH_BINARY)

        contours, _ = cv2.findContours(fg_mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

        for contour in contours:
            x, y, w, h = cv2.boundingRect(contour)
            cx, cy = x + w // 2, y + h // 2

            if w > max_vehicle_width or cv2.contourArea(contour) < 500:
                continue

            for i, line in enumerate(lines_positions):
                if crosses_line(cx, cy, line):
                    if cv2.contourArea(contour) < area_thresholds[i]:
                        continue

                    new_object = True
                    for obj_id in list(tracked_objects[i].keys()):
                        prev_cx, prev_cy, last_seen = tracked_objects[i][obj_id]
                        if abs(cx - prev_cx) < 50 and abs(cy - prev_cy) < 50:
                            tracked_objects[i][obj_id] = (cx, cy, frame_count)
                            new_object = False
                            break

                    if new_object:
                        tracked_objects[i][object_ids[i]] = (cx, cy, frame_count)
                        vehicle_count[i] += 1
                        object_ids[i] += 1

        for i, _ in enumerate(lines_positions):
            expired_ids = [
                obj_id for obj_id, (_, _, last_seen) in tracked_objects[i].items()
                if frame_count - last_seen > object_expiry
            ]
            for obj_id in expired_ids:
                del tracked_objects[i][obj_id]

        for i, line in enumerate(lines_positions):
            if 2 <= i <= 6:
                label = f"Carril {i - 1}: {vehicle_count[i]}"
            else:
                label = f"Via {i + 1}: {vehicle_count[i]}"

            cv2.line(frame, (line["x1"], line["y1"]), (line["x2"], line["y2"]), (0, 255, 0), 2)
            cv2.putText(frame, label, (line["x1"], line["y1"] - 20),
                        cv2.FONT_HERSHEY_SIMPLEX, 0.6, (255, 255, 255), 2)

        cv2.imshow("Vehicle Counting", frame)

        if cv2.waitKey(30) & 0xFF == ord('q'):
            break

    cap.release()
    cv2.destroyAllWindows()

for i, count in enumerate(vehicle_count):
    if 2 <= i <= 6:
        print(f"Carril {i - 1}: {count} vehículos contados.")
    else:
        print(f"Via {i + 1}: {count} vehículos contados.")

Via 1: 5 vehículos contados.
Via 2: 5 vehículos contados.
Carril 1: 10 vehículos contados.
Carril 2: 15 vehículos contados.
Carril 3: 4 vehículos contados.
Carril 4: 10 vehículos contados.
Carril 5: 6 vehículos contados.
Via 8: 14 vehículos contados.


## **Código completo - Vídeo en bucle**

Este código incluye modificaciones clave para garantizar que el video se ejecute en un **bucle continuo**. A continuación, se describen los principales cambios:

1. **Uso de la variable `exit_program`**:
   - Se introduce la variable `exit_program` como una bandera para controlar si el bucle principal debe detenerse. 
   - Inicialmente, se establece en `False`, lo que permite que el programa continúe ejecutándose.

2. **Bucle principal (`while not exit_program`)**:
   - El programa se ejecuta dentro de un bucle principal que reinicia el video automáticamente cuando este llega al final.
   - Si el usuario presiona la tecla `q`, la variable `exit_program` cambia a `True`, lo que rompe el bucle y termina la ejecución.

3. **Bucle interno para procesamiento de frames**:
   - Este bucle procesa cada frame del video utilizando OpenCV.
   - Al llegar al final del video (`if not ret`), el bucle interno se rompe, liberando los recursos del video (`cap.release()`) y reiniciando la reproducción.

4. **Salida controlada con la tecla `q`**:
   - Dentro del bucle interno, se captura la entrada del teclado con `cv2.waitKey(30)`.
   - Si se detecta que el usuario presionó `q`, se cambia la bandera `exit_program` a `True` y se rompe el bucle interno, lo que termina la ejecución completa del programa.

Con estas modificaciones, el programa reproduce el video en bucle hasta que el usuario decide salir manualmente presionando la tecla `q`.

In [ ]:
import cv2 

video_path = 'tráfico01.mp4'

lines_positions = [
    {"x1": 480, "y1": 800, "x2": 570, "y2": 800},
    {"x1": 600, "y1": 800, "x2": 700, "y2": 800},
    {"x1": 990, "y1": 750, "x2": 1090, "y2": 750},
    {"x1": 1280, "y1": 890, "x2": 1500, "y2": 890},
    {"x1": 1200, "y1": 660, "x2": 1260, "y2": 660},
    {"x1": 1450, "y1": 750, "x2": 1550, "y2": 750},
    {"x1": 1650, "y1": 790, "x2": 1800, "y2": 790},
    {"x1": 1500, "y1": 450, "x2": 1800, "y2": 450},
]

area_thresholds = [400, 500, 700, 800, 700, 700, 500, 600]
max_vehicle_width = 400
object_expiry = 16

def crosses_line(cx, cy, line):
    return line["x1"] <= cx <= line["x2"] and abs(cy - line["y1"]) <= 10

# Variable de control para salir del bucle principal
exit_program = False

while not exit_program:  # Bucle principal
    vehicle_count = [0] * len(lines_positions)  # Reinicia el conteo de vehículos.
    tracked_objects = [{} for _ in range(len(lines_positions))]  # Reinicia el seguimiento de objetos.
    object_ids = [0] * len(lines_positions)  # Reinicia los identificadores únicos de objetos.

    cap = cv2.VideoCapture(video_path)  # Carga el video.

    if not cap.isOpened():  # Verifica si el video se cargó correctamente.
        print("Error: No se puede abrir el video.")
        break  # Finaliza si no se puede cargar el video.

    background_subtractor = cv2.createBackgroundSubtractorMOG2(history=500, varThreshold=50, detectShadows=False)
    frame_count = 0

    while True:  # Bucle interno para procesar cada frame del video.
        ret, frame = cap.read()
        if not ret:  # Si no se puede leer un frame (final del video):
            cap.release()  # Libera el recurso del video.
            break  # Rompe el bucle interno para reiniciar el video.

        frame_count += 1
        gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
        fg_mask = background_subtractor.apply(gray)
        fg_mask = cv2.morphologyEx(fg_mask, cv2.MORPH_CLOSE, cv2.getStructuringElement(cv2.MORPH_RECT, (5, 5)))
        _, fg_mask = cv2.threshold(fg_mask, 254, 255, cv2.THRESH_BINARY)

        contours, _ = cv2.findContours(fg_mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

        for contour in contours:
            x, y, w, h = cv2.boundingRect(contour)
            cx, cy = x + w // 2, y + h // 2

            if w > max_vehicle_width or cv2.contourArea(contour) < 500:
                continue

            for i, line in enumerate(lines_positions):
                if crosses_line(cx, cy, line):
                    if cv2.contourArea(contour) < area_thresholds[i]:
                        continue

                    new_object = True
                    for obj_id in list(tracked_objects[i].keys()):
                        prev_cx, prev_cy, last_seen = tracked_objects[i][obj_id]
                        if abs(cx - prev_cx) < 50 and abs(cy - prev_cy) < 50:
                            tracked_objects[i][obj_id] = (cx, cy, frame_count)
                            new_object = False
                            break

                    if new_object:
                        tracked_objects[i][object_ids[i]] = (cx, cy, frame_count)
                        vehicle_count[i] += 1
                        object_ids[i] += 1

        for i, _ in enumerate(lines_positions):
            expired_ids = [
                obj_id for obj_id, (_, _, last_seen) in tracked_objects[i].items()
                if frame_count - last_seen > object_expiry
            ]
            for obj_id in expired_ids:
                del tracked_objects[i][obj_id]

        for i, line in enumerate(lines_positions):
            if 2 <= i <= 6:
                label = f"Carril {i - 1}: {vehicle_count[i]}"
            else:
                label = f"Via {i + 1}: {vehicle_count[i]}"

            cv2.line(frame, (line["x1"], line["y1"]), (line["x2"], line["y2"]), (0, 255, 0), 2)
            cv2.putText(frame, label, (line["x1"], line["y1"] - 20),
                        cv2.FONT_HERSHEY_SIMPLEX, 0.6, (255, 255, 255), 2)

        cv2.imshow("Vehicle Counting", frame)

        key = cv2.waitKey(30) & 0xFF  # Captura la tecla presionada.
        if key == ord('q'):  # Si se presiona 'q':
            exit_program = True  # Cambia el estado para salir del bucle principal.
            break  # Rompe el bucle interno para salir completamente.

    cap.release()
    cv2.destroyAllWindows()

for i, count in enumerate(vehicle_count):
    if 2 <= i <= 6:
        print(f"Carril {i - 1}: {count} vehículos contados.")
    else:
        print(f"Via {i + 1}: {count} vehículos contados.")